In [1]:
import numpy as np 
import pickle as pkl 
import os

# Create 32x32 data with labels 

In [2]:
destination_directory = '/home/yiwei/data/core/'
pkl_file = open(os.path.join(destination_directory, 'paths.pkl') , 'rb') 
paths = pkl.load(pkl_file) 

In [3]:
imgs = np.load(os.path.join(destination_directory, 'core50_imgs.npz'))['x'] 

In [4]:

import re

pattern = r'C_(\d+)_(\d+)_(\d+)\.png'

def parse(filename):
    match = re.match(pattern, filename)

    if match:
        session_num = match.group(1)
        obj_num = match.group(2)
        frame_seq_id = match.group(3)
        return session_num, obj_num, frame_seq_id
    else:
        print("No match found.")
        # AssertionError()

In [5]:
filter = r's(\d+)/o(\d+)/'
def filter_dir(filename):
    filename = re.sub(filter, '', filename)
    return filename

In [6]:
def object2class(obj_id):
    return (int(obj_id) - 1) // 5

In [7]:
session_ids, obj_ids, frame_ids, class_ids = [], [], [], []
for path in paths:
    session_id, obj_id, frame_id = parse(filter_dir(path))
    session_ids.append(int(session_id) - 1)
    obj_ids.append(int(obj_id) - 1)
    frame_ids.append(int(frame_id))
    class_ids.append(object2class(obj_id))

## Resize to 32 x 32

In [8]:
from PIL import Image
import torchvision.transforms as T

In [43]:
img = Image.fromarray(imgs[0])

In [9]:
data_32x32 = []
for img in imgs:
    img = Image.fromarray(img)
    resized_imgs = T.Resize(size=32)(img)
    resized_imgs = np.asarray(resized_imgs)
    data_32x32.append(resized_imgs)

## Save to pkl

In [10]:
core = {
    'data': data_32x32,
    'session': session_ids,
    'object': obj_ids,
    'frame': frame_ids,
    'category': class_ids
}


In [11]:
with open('core_data.pkl', 'wb') as f:
    pkl.dump(core,f)

In [56]:
from PIL import Image
print(class_ids[10])
img = Image.fromarray(data[10])
img.save('demo.png')

0


In [2]:
fo = open('core_data.pkl', 'rb')
core = pkl.load(fo)

In [61]:
cats = [ 6, 8 ]
sessions = [i for i in range(11)]


In [57]:
obj_labels = np.asarray(core['object'])
se_labels = np.asarray(core['session'])
cat_labels = np.asarray(core['category'])

In [9]:
data = np.asarray(core['data'])

In [10]:
data.shape

(164866, 32, 32, 3)

In [58]:
def get_subset(dataset_labels, targets, dataset_indices, sample=0):
    '''
    Find indices of subsets where labels match with targets
    '''
    results = []
    for target in targets:
        subset_indicies = dataset_indices[dataset_labels==target]
        if sample > 0:
            subset_indicies = np.random.choice(subset_indicies, 30, replace=False)
        results.append(subset_indicies)
    results = np.concatenate(results)
    return results

In [62]:
indices = np.arange(len(data))

sampled_cat_indices = []
for cat in cats:
    cat_indices = indices[cat_labels==cat]
    cat_session_labels = se_labels[cat_indices]

    objects = [i + cat * 5 for i in range(5)]

    sampled_session_indices = []
    for session in sessions: # session labels given a cat

        session_indices = cat_indices[cat_session_labels==session]
        session_obj_labels = obj_labels[session_indices]

        # sampled_object_indices = []
        
        subset_indices = get_subset(session_obj_labels, objects, session_indices, sample=30) # obj labels given a session
        sampled_session_indices.append(subset_indices)
    
    sampled_session_indices = np.concatenate(sampled_session_indices, axis = 0)

    sampled_cat_indices.append(sampled_session_indices)

sampled_cat_indices = np.concatenate(sampled_cat_indices, axis = 0)

sampled_data = data[sampled_cat_indices]

In [63]:
np.unique(obj_labels[sampled_cat_indices])

array([30, 31, 32, 33, 34, 40, 41, 42, 43, 44])